In [37]:
import pyawr.mwoffice as mwo

import numpy as np
from numpy.polynomial.polynomial import polyfit

import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation 

from scipy.signal import savgol_filter

#***PICK ONE***
#Agg rendering embedded in a Jupyter widget. (inline) Requires ipympl:
# %matplotlib ipympl 
#Agg rendering to a Tk canvas (new window) Requires TkInter:
%matplotlib tk 

In [2]:
awrde = mwo.CMWOffice() #Create awrde object
awrde.Project.Simulator.Analyze() #Invoke circuit simulator analysis

In [3]:
def reset_freqs (l_bnd=4e9, u_bnd=8e9, steps=10000):
    '''Reset the MWO project frequencies.

    returns an array running from 
    lower_bound to upper_bound in steps steps.'''
    
    awrde.Project.Frequencies.Clear() # clear the frequencies specified for the project
    freq_arr = np.linspace(l_bnd, u_bnd, steps)
    awrde.Project.Frequencies.AddMultiple(freq_arr) # add the frequencies in the passed frequency array

    awrde.Project.Simulator.Analyze() # must run this every time after setting or changing things in MWO project!

    return freq_arr

In [4]:
def set_circ_params(circ_name='Inductor_Subcircuit', get_params=False, **kwargs): 
    '''Sets specified circuit parameters in circuit called circ_name.
    

    If you attempt to pass a parameter that doesn't exit in the circuit called circ_name, 
    MWO will throw an error.

    Returns a dictionary with the new values.
    '''

    passed_circ_param_vals = { # dictionary of subcircuit element parameter values (e.g. the value of the capacitor (element's) capacitance (parameter.))
        'CAP': awrde.Project.Schematics(circ_name).Elements(1).Parameters(2).ValueAsDouble,
        'IND': awrde.Project.Schematics(circ_name).Elements(2).Parameters(2).ValueAsDouble,
        'RES': awrde.Project.Schematics(circ_name).Elements(3).Parameters(2).ValueAsDouble
    }
    passed_circ_params = { # dictionary of subcircuit component 
        'CAP': awrde.Project.Schematics(circ_name).Elements(1).Parameters(2),
        'IND': awrde.Project.Schematics(circ_name).Elements(2).Parameters(2),
        'RES': awrde.Project.Schematics(circ_name).Elements(3).Parameters(2)
    }

    # print("DEBUG: Setting:" + kwargs.__str__())

    new_circ_param_vals = {**passed_circ_param_vals, **kwargs} # in the case of duplicate keys, only the later key-value pair is preserved
    # print("DEBUG: new parameters:" + new_circ_param_vals.__str__())

    for i, value in enumerate(new_circ_param_vals.values()):
        list(passed_circ_params.values())[i].ValueAsDouble = value # in python3, dict.keys(), .values(), and .items() return dynamically changing view objects, but not the objects themselves. Hence, list().

        # print("DEBUG: " + list(passed_circ_param_vals)[i] + " set to " + str(value))
    
    if not(get_params):
        awrde.Project.Simulator.Analyze() # must run this every time after setting or changing things in MWO project!

    return new_circ_param_vals

In [32]:
# ***********************************************
# *                Plotting S11                 *
# ***********************************************

def plot_graph(graph_name='Parallel', show_max=False, show_params=False, *args, **kwargs):
    graph = awrde.Project.Graphs(graph_name)
    meas = graph.Measurements[0]

    def animate_smooth(n_frm): #Animate method for FuncAnimation
        xs = np.asarray(meas.XValues)
        ys = np.asarray(meas.YValues(1))
        ys_savgol = savgol_filter(ys, window_length=99, polyorder=3)
        dys_savgol = savgol_filter(ys, window_length=99, polyorder=3, deriv=1)
        abs_dys_savgol = np.abs(dys_savgol)

        axs_smooth[0].cla() # clear the axes
        axs_smooth[1].cla() #

        if show_max:
            max_ind = np.argmax(abs_dys_savgol)
            max_x = xs[max_ind]
            max_y = ys[max_ind]
            max_dy = dys_savgol[max_ind]
            axs_smooth[0].plot(max_x, max_y, 'ro')

            label = "({:.4g}, 1.0+{:.4g},); deriv.={:.4g})".format(max_x, 1.0-max_y, max_dy)
            axs_smooth[0].annotate(label, (max_x, max_y), textcoords='offset points', xytext=(5,2), ha='left', size=12)

        if show_params:
            # NOTE: Only will work with values for the inductor subcircuit
            circ_param_string = set_circ_params(get_params=True)
            # axs_smooth[0].set_title(circ_param_string.__str__())
            axs_smooth[0].text(0.01, 0.01, circ_param_string, va='bottom', ha='left',transform=axs_smooth[0].transAxes, color='black', fontsize=10)

        axs_smooth[0].plot(xs, ys_savgol, 'b') # applying a savgol filer
        axs_smooth[0].set_ylabel('|S11|')
        axs_smooth[1].plot(xs, abs_dys_savgol, 'r') # applying a savgol filter, with differentiation
        axs_smooth[1].set_xlabel('frequency')
        axs_smooth[1].set_ylabel('deriv. of |S11|')
        # axs_smooth[0].plot(xs, ys, 'r')
        # axs_smooth[1].plot(xs, abs_dys, 'b')

    # TODO: Get zoom to work?
    fig_smooth, axs_smooth = plt.subplots(2, 1, sharex='all')
    
    #TODO: Figure out this blit thing
    ani = FuncAnimation(fig_smooth, animate_smooth, interval=500, blit=False, **kwargs) # create animation object. blit=True is for smoother animations, only changed data should be updated.
    fig_smooth.suptitle(graph_name)
    plt.show()
    return ani # Important! Without this, ani goes out of scope and the animation gets garbage collected!


# ***********************************************
# *           Plotting S11: Parallel            *
# ***********************************************

# graph_name = 'Parallel'
# ani1 = plot_graph(graph_name, show_max=True) 


# ***********************************************
# *            Plotting S11: Series             *
# ***********************************************

# graph_name = 'Series'
# ani2 = plot_graph(graph_name, show_max=True)

In [6]:
def get_measurements(graph_name='Parallel'):
    graph = awrde.Project.Graphs(graph_name)
    meas = graph.Measurements[0]
    num_pts = meas.XPointCount
    xs = ys = np.zeros(num_pts)

    xs = np.asarray(meas.XValues)
    ys = np.asarray(meas.YValues(1))
    ys_savgol = savgol_filter(ys, window_length=51, polyorder=3)
    dys_savgol = savgol_filter(ys, window_length=51, polyorder=3, deriv=1)
    abs_dys_savgol = np.abs(dys_savgol)

    # print("DEBUG: Got xs, ys, abs_dys_savgol = " + (xs, ys, abs_dys_savgol).__str__() + ".")

    return (xs, abs_dys_savgol, ys_savgol)

In [7]:
def plot_3d(x, y, s11_arr_2d):
    # from mpl_toolkits import mplot3d

    xs, ys = np.meshgrid(x, y)
    zs = s11_arr_2d

    fig = plt.figure()
    ax3d = plt.axes(projection='3d')
    ax3d.contour(xs, ys, zs, 40, cmap='binary')
    # ax3d.plot_surface(xs, ys, zs, rstride=1, cstride=1, cmap='viridis', edgecolor='none')
    ax3d.set_xlabel('frequency')
    ax3d.set_ylabel('capacitance')
    ax3d.set_zlabel('S11')
    plt.show()

    # print('DEBUG: ' + (np.shape(xs), np.shape(ys), np.shape(zs)).__str__())
    

In [8]:
def check_gaussian_fit(freqs, s11s):
    p, res, _, _, _ = np.polyfit(freqs, s11s, 1, full=True)
    print(res)

In [39]:
# ***********************************************
# *    Specifiying the sweep of a parameter:    *
# ***********************************************

# ---------------USER-SETTABLE:-------------- #
# If a parameter is missing from the dictionary it will not be reset.
# If you want to keep something constant then comment it out.
# If you want to set it to a differnt constant then set lower_bound = upper_bound. 
# If you only want to change one variable at a time then... uhh... stay tuned.
#'COMPONENT' : (lower_bound, upper_bound)    
sample_circ_param_bounds = {
    'CAP': (1e-15,1e-14),
    # 'IND': (0,1),
    'RES': (1000000000,1000000000)
}

resolution = 5 # sets the number of parameter values to simulate (i.e. the resolution.)
graph_names = ('Parallel', 'Series') # choose the graph names to simulate. Don't delete the comma.
show_3d_plot = False # set this to true if you want to see a 3d graph
# ------------------------------------------- #

sample_circ_param_arrays = dict.fromkeys(sample_circ_param_bounds)

keys = sample_circ_param_arrays.keys()
bnds = sample_circ_param_bounds.values()
arrs = list(np.linspace(*bnds_tuple, resolution) for bnds_tuple in bnds)

sample_circ_param_arrays.update(zip(keys, arrs))

# print("DEBUG: " + sample_circ_param_arrays.__str__())
 

# ***********************************************
# * Running the sumulation with set parameters: *
# ***********************************************

ani_p = ani_s = None # prevents deletion of AnimationFunction object when loop concludes.

for graph_name in graph_names:
    # This is obscenely inefficient because MWO analyzes both the parallel and the series circuit anyways

    # print('DEBUG: graph_name is' + graph_name)

    freqs = reset_freqs()
    s11_derivs_arr = np.zeros((resolution, len(freqs)))

    max_x = 0
    max_s11_derivs = max_freqs = np.zeros_like(freqs)

    # FIXME: This loop actually changes all the elements simultaneously. Add some logic to do it iteratively...
    for ind in range(resolution):
        indiv_vals = list(arr[ind] for arr in arrs)
        ele_val_dict = dict(zip(keys, indiv_vals))
        # TODO: This is probably not great, first creating a dictionary then unpacking it immediately...
        set_circ_params(**ele_val_dict)
        # print('DEBUG: ' + ele_val_dict.__str__())

        freqs, s11_derivs, s11s = get_measurements(graph_name)
        max_ind = np.argmax(s11_derivs)
        max_freq = freqs[max_ind]
        max_x = max(max_freq, max_x)

        if show_3d_plot:
            s11_derivs_arr[ind] = s11_derivs
            # print('DEBUG: Set s11_derivs_arr[{}] to {}.'.format(ind, s11_derivs))

        # TODO: test this
        if max_x == max_freq:
            max_s11_derivs = s11_derivs
            max_freqs = freqs 


    if show_3d_plot:
        #NOTE: This is hardcoded and will only work for a CAP sweep for now!
        ys_3d = np.linspace(*(sample_circ_param_bounds.get('CAP')), resolution)
        plot_3d(freqs, ys_3d, s11_derivs_arr)

    else:
        # FIXME: hardcoded to prevent FuncAnimation object from going out of scope.
        if graph_name=="Parallel":
            reset_freqs(max_x - 1e9, max_x + 1e9)
            ani_p = plot_graph(graph_name, show_max=True, show_params=True, frames=1)
        elif graph_name=="Series":
            reset_freqs(max_x - 1e9, max_x + 1e9)
            ani_s = plot_graph(graph_name, show_max=True, show_params=True, frames=1)